In [100]:
import os
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from neurokit2 import eda_process

plt.style.use('ggplot')

time_middle_file_path = 'middle-files/'
eda_output_path = 'eda-middle-data/'

p_index = 8

def get_key_timestamps(target, duration_ms, df, df_key='timestamp'):
    target_after = target + duration_ms

    idx_target = df[df_key].searchsorted(target, 'right') - 1
    idx_after = df[df_key].searchsorted(target_after, 'left')

    timestamp_target = df[df_key].iloc[idx_target] if idx_target >= 0 else 0
    timestamp_after = df[df_key].iloc[idx_after] if idx_after < len(df) else 0

    return timestamp_target, timestamp_after



In [107]:

participant_key = 'P' + str(p_index + 1).zfill(2)

df_time = pd.read_csv(f'{time_middle_file_path}{participant_key}_foreground_background_differences.csv')
df_eda = pd.read_csv(f'dataset/{participant_key}/EDA.csv')

# print(f'dataset/{participant_key}/EDA.csv')

print(df_eda[df_eda['resistance'] == 0])

# transform from ohm to microsiemens (Note: the original paper said it is kilo-ohm but seems wrong...)
df_eda['conductance'] = 1 / df_eda['resistance'] * 1000

# print(pd.Series(df_eda['resistance'].tolist()).describe())


# potential: transform to z-score?
df_eda['conductance'] = stats.zscore(df_eda['conductance'].tolist())



             timestamp  resistance
170009   1557307469035           0
170010   1557307469252           0
170012   1557307469633           0
170030   1557307473219           0
170065   1557307480192           0
170126   1557307492345           0
448510   1557473413038           0
448511   1557473413249           0
448512   1557473413454           0
448513   1557473413671           0
448514   1557473413849           0
448515   1557473414036           0
448516   1557473414235           0
448517   1557473414450           0
448518   1557473414638           0
448519   1557473414842           0
448520   1557473415030           0
456539   1557475012576           0
456541   1557475012963           0
745716   1557657304122           0
745717   1557657304316           0
745718   1557657304517           0
847549   1557678991435           0
847550   1557678991635           0
847552   1557678992032           0
847553   1557678992231           0
847554   1557678992429           0
853558   15577047533

C:\Users\xchen\.virtualenvs\INFOMQNM-Project-AQz3gs5P\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\xchen\.virtualenvs\INFOMQNM-Project-AQz3gs5P\Lib\site-packages\scipy\stats\_stats_py.py:3100: RuntimeWarning: invalid value encountered in subtract
  z = (scores - mn) / std


In [102]:
# filter out use time that is less than 1 minute
loc_time = df_time.loc[df_time.time_difference > 60 * 1000]

res = pd.DataFrame(columns=['foreground_time', 'max_amplitude'])
for index, row in loc_time.iterrows():
    foreground_time = row['foreground_time']
    background_time = row['background_time']

    (ts_target, ts_after) = get_key_timestamps(foreground_time, 60 * 1000, df_eda)

    if ts_target > 0 and ts_after > 0:
        df_analyse = df_eda.loc[(df_eda['timestamp'] >= ts_target) & (df_eda['timestamp'] <= ts_after)]

        # data is too short to get meaningful result
        if (len(df_analyse)) < 15 * 2:
            continue

        eda_res, info = eda_process(df_analyse['conductance'], 5, kwargs_phasic='SparsEDA')

        res.loc[len(res)] = [foreground_time, np.nanmax(info['SCR_Amplitude'])]

res.to_csv(eda_output_path + participant_key + '_eda_result.csv', index=False)
print(participant_key + ' done')

C:\Users\xchen\.virtualenvs\INFOMQNM-Project-AQz3gs5P\Lib\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 303 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\xchen\.virtualenvs\INFOMQNM-Project-AQz3gs5P\Lib\site-packages\neurokit2\eda\eda_clean.py:94: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  eda_signal = pd.DataFrame.pad(pd.Series(eda_signal))
C:\Users\xchen\.virtualenvs\INFOMQNM-Project-AQz3gs5P\Lib\site-packages\neurokit2\eda\eda_clean.py:105: NeuroKitWarning: EDA signal is sampled at very low frequency. Skipping filtering.
  warn(


KeyError: 0